# Knowledge Graph Embeddings

This notebook is about your first steps with knowledge graph embedding models in PyKEEN. This notebook is based on https://github.com/pykeen/pykeen/blob/master/notebooks/hello_world/hello_world.ipynb

You'll get to do the following:

1. train a model based on a small knowledge graph
2. evaluate how good it learned
3. turn it around and start making predictions.

The knowledge graph is provided inline as string and is originally based on the `Countries` dataset provided with Pykeen.


## Install Packages

In [ ]:
# Install packages if they're not already found
! pip install --upgrade pip
! python -c "import pykeen" || pip install git+https://github.com/pykeen/pykeen.git
! python -c "import wordcloud" || pip install wordcloud

## Imports

In [ ]:
import os

import matplotlib.pyplot as plt
import torch

import pykeen
from pykeen.datasets import Countries
from pykeen.pipeline import pipeline
from pykeen import predict
from pykeen.triples import TriplesFactory

import requests
import numpy as np

import pandas as pd

pd.set_option('display.max_columns', 100)

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

## Environment variables

In [ ]:
pykeen.env()

## The Knowledge Graph

The knowledge graph is provided inline as String to facilitate manipulation. You can add and delete triples as you wish. You have to separate subject, predicate and objects by tabs.

In the current KG, Switzerland has no information about which part of europe it is located in and its languages.

In [ ]:
data = """
albania	language	albanian
albania	language	macedonian
albania	language	greek
albania	locatedin	southern_europe
albania	neighbor	greece
albania	neighbor	kosovo
albania	neighbor	macedonia
albania	neighbor	montenegro
andorra	language	catalan
andorra	locatedin	southern_europe
andorra	neighbor	france
andorra	neighbor	spain
austria	language	croatian
austria	language	german
austria	language	hungarian
austria	language	slovene
austria	locatedin	western_europe
austria	neighbor	czechia
austria	neighbor	germany
austria	neighbor	hungary
austria	neighbor	italy
austria	neighbor	liechtenstein
austria	neighbor	slovakia
belarus	language	belarussian
belarus	language	russian
belarus	locatedin	eastern_europe
belarus	neighbor	latvia
belarus	neighbor	lithuania
belarus	neighbor	poland
belarus	neighbor	russia
belarus	neighbor	ukraine
belgium	language	dutch
belgium	language	french
belgium	language	german
belgium	locatedin	western_europe
belgium	neighbor	france
belgium	neighbor	germany
belgium	neighbor	luxembourg
belgium	neighbor	netherlands
bosnia_and_herzegovina	language	bosnian
bosnia_and_herzegovina	language	croation
bosnia_and_herzegovina	language	serbian
bosnia_and_herzegovina	locatedin	southern_europe
bosnia_and_herzegovina	neighbor	croatia
bosnia_and_herzegovina	neighbor	montenegro
bosnia_and_herzegovina	neighbor	serbia
bulgaria	language	bulgarian
bulgaria	locatedin	eastern_europe
bulgaria	neighbor	greece
bulgaria	neighbor	macedonia
bulgaria	neighbor	romania
bulgaria	neighbor	serbia
bulgaria	neighbor	turkey
croatia	language	croatian
croatia	locatedin	southern_europe
croatia	neighbor	bosnia_and_herzegovina
croatia	neighbor	hungary
croatia	neighbor	montenegro
croatia	neighbor	serbia
croatia	neighbor	slovenia
cyprus	language	greek
cyprus	language	turkish
cyprus	locatedin	eastern_europe
cyprus	neighbor	united_kingdom
czechia	language	czech
czechia	locatedin	eastern_europe
czechia	neighbor	austria
czechia	neighbor	germany
czechia	neighbor	poland
czechia	neighbor	slovakia
denmark	language	danish
denmark	language	german
denmark	locatedin	northern_europe
denmark	neighbor	germany
estonia	language	estonian
estonia	locatedin	northern_europe
estonia	neighbor	latvia
estonia	neighbor	russia
finland	language	finnish
finland	language	swedish
finland	locatedin	northern_europe
finland	neighbor	norway
finland	neighbor	russia
finland	neighbor	sweden
france	language	french
france	locatedin	western_europe
france	neighbor	andorra
france	neighbor	belgium
france	neighbor	germany
france	neighbor	italy
france	neighbor	luxembourg
france	neighbor	monaco
france	neighbor	spain
france	neighbor	switzerland
germany	language	german
germany	locatedin	western_europe
germany	neighbor	austria
germany	neighbor	belgium
germany	neighbor	czechia
germany	neighbor	denmark
germany	neighbor	france
germany	neighbor	luxembourg
germany	neighbor	netherlands
germany	neighbor	poland
germany	neighbor	switzerland
gibraltar	language	english
gibraltar	language	spanish
gibraltar	locatedin	southern_europe
gibraltar	neighbor	spain
greece	language	greek
greece	locatedin	southern_europe
greece	neighbor	albania
greece	neighbor	bulgaria
greece	neighbor	macedonia
greece	neighbor	turkey
hungary	language	hungarian
hungary	locatedin	eastern_europe
hungary	neighbor	austria
hungary	neighbor	croatia
hungary	neighbor	romania
hungary	neighbor	serbia
hungary	neighbor	slovakia
hungary	neighbor	slovenia
hungary	neighbor	ukraine
iceland	language	icelandic
iceland	locatedin	northern_europe
ireland	locatedin	northern_europe
ireland	neighbor	united_kingdom
italy	language	italian
italy	locatedin	southern_europe
italy	neighbor	austria
italy	neighbor	france
italy	neighbor	san_marino
italy	neighbor	slovenia
italy	neighbor	switzerland
italy	neighbor	vatican_city
kosovo	language	albanian
kosovo	language	serbian
kosovo	locatedin	eastern_europe
kosovo	neighbor	albania
kosovo	neighbor	macedonia
kosovo	neighbor	montenegro
kosovo	neighbor	serbia
latvia	language	latvian
latvia	locatedin	northern_europe
latvia	neighbor	belarus
latvia	neighbor	estonia
latvia	neighbor	lithuania
latvia	neighbor	russia
liechtenstein	language	german
liechtenstein	locatedin	western_europe
liechtenstein	neighbor	austria
liechtenstein	neighbor	switzerland
lithuania	language	lithuanian
lithuania	locatedin	northern_europe
lithuania	neighbor	belarus
lithuania	neighbor	latvia
lithuania	neighbor	poland
lithuania	neighbor	russia
luxembourg	language	french
luxembourg	language	german
luxembourg	language	luxembourgish
luxembourg	locatedin	western_europe
luxembourg	neighbor	belgium
luxembourg	neighbor	france
luxembourg	neighbor	germany
macedonia	language	albanian
macedonia	language	macedonian
macedonia	locatedin	southern_europe
macedonia	neighbor	albania
macedonia	neighbor	bulgaria
macedonia	neighbor	greece
macedonia	neighbor	kosovo
macedonia	neighbor	serbia
malta	language	english
malta	language	maltese
malta	locatedin	southern_europe
moldova	language	romanian
moldova	locatedin	eastern_europe
moldova	neighbor	romania
moldova	neighbor	ukraine
monaco	language	french
monaco	language	italian
monaco	locatedin	western_europe
monaco	neighbor	france
montenegro	language	montenegrin
montenegro	language	albanian
montenegro	language	bosnian
montenegro	language	croatian
montenegro	language	serbian
montenegro	locatedin	southern_europe
montenegro	neighbor	albania
montenegro	neighbor	bosnia_and_herzegovina
montenegro	neighbor	croatia
montenegro	neighbor	kosovo
montenegro	neighbor	serbia
netherlands	language	dutch
netherlands	locatedin	western_europe
netherlands	neighbor	belgium
netherlands	neighbor	germany
norway	language	norwegian
norway	locatedin	northern_europe
norway	neighbor	finland
norway	neighbor	russia
norway	neighbor	sweden
poland	language	polish
poland	locatedin	eastern_europe
poland	neighbor	belarus
poland	neighbor	czechia
poland	neighbor	germany
poland	neighbor	lithuania
poland	neighbor	russia
poland	neighbor	slovakia
poland	neighbor	ukraine
portugal	language	portuguese
portugal	locatedin	southern_europe
portugal	neighbor	spain
romania	language	romanian
romania	locatedin	eastern_europe
romania	neighbor	bulgaria
romania	neighbor	hungary
romania	neighbor	moldova
romania	neighbor	serbia
romania	neighbor	ukraine
russia	language	russian
russia	locatedin	eastern_europe
russia	neighbor	belarus
russia	neighbor	estonia
russia	neighbor	finland
russia	neighbor	latvia
russia	neighbor	lithuania
russia	neighbor	norway
russia	neighbor	poland
russia	neighbor	ukraine
san_marino	language	italian
san_marino	locatedin	southern_europe
san_marino	neighbor	italy
serbia	language	serbian
serbia	locatedin	southern_europe
serbia	neighbor	bosnia_and_herzegovina
serbia	neighbor	bulgaria
serbia	neighbor	croatia
serbia	neighbor	hungary
serbia	neighbor	kosovo
serbia	neighbor	macedonia
serbia	neighbor	montenegro
serbia	neighbor	romania
slovakia	language	slovak
slovakia	locatedin	central_europe
slovakia	neighbor	austria
slovakia	neighbor	czechia
slovakia	neighbor	hungary
slovakia	neighbor	poland
slovakia	neighbor	ukraine
slovenia	language	hungarian
slovenia	language	italian
slovenia	language	slovene
slovenia	locatedin	southern_europe
slovenia	neighbor	austria
slovenia	neighbor	croatia
slovenia	neighbor	hungary
slovenia	neighbor	italy
spain	language	spanish
spain	language	catalan
spain	locatedin	southern_europe
spain	neighbor	andorra
spain	neighbor	france
spain	neighbor	gibraltar
spain	neighbor	morocco
spain	neighbor	portugal
sweden	language	swedish
sweden	language	finnish
sweden	locatedin	northern_europe
sweden	neighbor	finland
sweden	neighbor	norway
switzerland	neighbor	france
switzerland	neighbor	germany
switzerland	neighbor	italy
switzerland	neighbor	liechtenstein
turkey	language	turkish
turkey	neighbor	bulgaria
turkey	neighbor	greece
ukraine	language	ukrainian
ukraine	language	russian
ukraine	locatedin	eastern_europe
ukraine	locatedin	eastern_europe
ukraine	neighbor	belarus
ukraine	neighbor	hungary
ukraine	neighbor	moldova
ukraine	neighbor	poland
ukraine	neighbor	romania
ukraine	neighbor	russia
ukraine	neighbor	slovakia
united_kingdom	language	english
united_kingdom	locatedin	northern_europe
united_kingdom	neighbor	ireland
vatican_city	language	latin
vatican_city	language	italian
vatican_city	locatedin	southern_europe
vatican_city	neighbor	italy
france	politicalalliancewith	germany
germany	politicalalliancewith	france
germany	politicalalliancewith	netherlands
malta	politicalalliancewith	estonia
ireland	politicalalliancewith	bulgaria
czechia	politicalalliancewith	romania
cyprus	politicalalliancewith	luxembourg
croatia	politicalalliancewith	slovenia
lithuania	politicalalliancewith	slovakia
portugal	politicalalliancewith	sweden
latvia	politicalalliancewith	italy
belgium	politicalalliancewith	hungary
romania	tradepartner	sweden
malta	tradepartner	netherlands
estonia	tradepartner	bulgaria
croatia	tradepartner	slovenia
cyprus	tradepartner	slovakia
lithuania	tradepartner	ireland
germany	tradepartner	denmark
finland	tradepartner	portugal
latvia	tradepartner	hungary
greece	tradepartner	italy
slovakia	tradepartner	luxembourg
belgium	tradepartner	czechia
bulgaria	tradepartner	croatia
""".strip()

### Triple Factories

The data is to be split up in a training set and a testing set and provided as triple factories.

In [ ]:
def get_tf(triples):
  triples = np.array([triple.split("\t") for triple in triples.split("\n")])
  tf = TriplesFactory.from_labeled_triples(triples=triples)
  return tf

data_tf = get_tf(data)
train_tf, test_tf = data_tf.split()

## Train a Model

We use PyKeen's `pipeline` to learn a knowledge graph embedding from the dataset.

More tutorials on training your first model can be found [here](https://pykeen.readthedocs.io/en/latest/tutorial/first_steps.html).

In [ ]:
result = pipeline(
    training=train_tf,
    testing=test_tf,
    model="PairRE",

    # Training configuration
    training_kwargs=dict(
        num_epochs=200,
        use_tqdm_batch=False,
    ),
    # Runtime configuration
    random_seed=1235,
)

## Internals

Save the trained model (which contains the triples and all of the learned parameters), the results from training, and all of the experimental metadata.

In [ ]:
save_location = "results"  # this directory
result.save_to_directory(save_location)
os.listdir(save_location)

In [ ]:
# How to look at the model
model = result.model
model

In [ ]:
# How to look at the triples
tf = result.training
tf

In [ ]:
tf.num_entities, tf.num_relations, tf.num_triples

## Evaluating the Results

Check that the loss is going down (and not oscillating) by plotting in the notebook.

In [ ]:
result.plot_losses()
plt.show()

Look at the result. These are pretty tricky to interpret, so remember:

- adjusted mean rank is between [0, 2]. Closer to 0 is better!
- mean rank is a positive integer, with a bound based on the number of entities. Closer to 0 is better!
- hits@k is reported between [0, 1] and interpreted as a percentage. Closer to 1 is better!

In [ ]:
result.metric_results.to_df()

## Turn it around: make predictions

It's very difficult to interpret KGEMs statistically, so it's best to sort order the predictions by their scores. All interaction functions in PyKEEN have been implemented such that the higher the score (or less negative the score), the more likely a triple is to be true.

Before making any predictions, we're goign to show some word clouds of the entities and relations in the Nations dataset, with size corresponding to frequency of appearance in triples.

In [ ]:
tf.entity_word_cloud()

In [ ]:
tf.relation_word_cloud()

## Predictions

### Where is Switzerland located?

In [ ]:
predict.predict_target(model=model, head="switzerland", relation="locatedin", triples_factory=tf).add_membership_columns(testing=test_tf,training=train_tf).df.head(20)

###Nachvollziehen warum predictions gemacht werden

germany, france, italy, liechtenstein haben das Land switzerand als neighbour in einem triple.
France, germany und liechtenstein sind located_in western_europe.

Daher wird das Model predicten, dass switzerland auch in central_europe liegt.


### What languages are spoken in Switzerland? (in addition to what is asserted in the KG)?

In [ ]:
# Automatically filter out non-novel predictions (e.g. in training or in testing is True)
predict.predict_target(model=model, head="switzerland", relation="language", triples_factory=tf).filter_triples(test_tf,train_tf).df.head(20)

###Nachvollziehen warum predictions gemacht werden

germany, france, italy haben das Land switzerand als neighbour in einem triple.

Weil diese Länder in relativ vielen Triples vorkommen, wird das Modell annehmen, dass in der Schweiz german/french/italian gesprochen wird.

Luxembourg hat wiederum germany und france als neighbour in einem Triple.
Daher wird das Modell annehmen, dass in switzerland auch luxembourgish gesprochen wird.

### Which countries are located in Western Europe? (in addition to those asserted in the KG)



In [ ]:
predict.predict_target(model=model, tail="western_europe", relation="locatedin", triples_factory=tf).filter_triples(test_tf,train_tf).df.head(20)

###Nachvollziehen warum predictions gemacht werden

Switzerland wird in western_europe predicted, weil Länder (france, germany, liechtenstein) die switzerland als neighbour haben und in western_europe locoated sind viele triples haben. Italy (switzerland als neighbor) liegt allerdings in southern europe.

### Erste Erweiterung des KG um folgende relation:
x tradepartner y

In [ ]:
predict.predict_target(model=model, head="austria", relation="tradepartner", triples_factory=tf).filter_triples(test_tf,train_tf).df.head(20)

### Zweite Erweiterung des KG um folgende relation:
x politicalalliancewith y


In [ ]:
predict.predict_target(model=model, head="austria", relation="politicalalliancewith", triples_factory=tf).filter_triples(test_tf,train_tf).df.head(20)